In [1]:
!pip install tensorflow efficientnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.1 MB/s eta 0:00:00


In [2]:
import os
import random
import shutil
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tarfile
import tensorflow as tf
import os
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tarfile
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [3]:
# !rm -r Images

In [4]:
# !rm -r test

In [5]:
# !rm -r train

In [6]:
!wget http://groups.csail.mit.edu/vision/LabelMe/NewImages/indoorCVPR_09.tar

--2023-07-05 05:53:11--  http://groups.csail.mit.edu/vision/LabelMe/NewImages/indoorCVPR_09.tar
Resolving groups.csail.mit.edu (groups.csail.mit.edu)... 128.30.2.44
Connecting to groups.csail.mit.edu (groups.csail.mit.edu)|128.30.2.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2592010240 (2.4G) [application/x-tar]
Saving to: ‘indoorCVPR_09.tar’

indoorCVPR_09.tar   100%[===================>]   2.41G  70.9MB/s    in 35s     

2023-07-05 05:53:46 (70.2 MB/s) - ‘indoorCVPR_09.tar’ saved [2592010240/2592010240]



In [7]:
with tarfile.open('/content/indoorCVPR_09.tar', 'r') as tar:
    tar.extractall()

In [8]:
dataset_path = '/content/Images'

images_folder = os.listdir('/content/Images')

dest_path = '/content'

train_ratio = 0.8

for folder in images_folder:
  # ['Gym2_000.jpg', 'gimnasio_36_08_flickr.jpg', ... ]
  folder_images_list = os.listdir(f'{dataset_path}/{folder}')
  train_images, test_images = train_test_split(folder_images_list, test_size=(1 - train_ratio), random_state=42)

  for image in train_images:
    src = os.path.join(dataset_path, folder, image)
    dest = os.path.join(dest_path, "train", folder, image)
    # print(src,dest)

    try:
        shutil.copy(src, dest)
    except IOError as io_err:
        os.makedirs(os.path.dirname(dest))
        shutil.copy(src, dest)

  for image in test_images:
    src = os.path.join(dataset_path, folder, image)
    dest = os.path.join(dest_path, "test", folder, image)
    # print(src,dest)

    try:
        shutil.copy(src, dest)
    except IOError as io_err:
        os.makedirs(os.path.dirname(dest))
        shutil.copy(src, dest)



In [9]:
# !pip install -U efficientnet

In [10]:
import efficientnet.keras as efn


# Load and preprocess your dataset
# Assuming your dataset is structured in separate folders for each class,
# where each folder contains images belonging to that class.
train_dir = '/content/train'
validation_dir = '/content/test'
image_size = (224, 224)  # Adjust the size according to your requirements



In [11]:
!mv "/content/test/classroom/Imagen_008.jpg" "/content"
!mv "/content/test/gym/gimnasio_74_01_flickr.jpg" "/content"
!mv "/content/test/lobby/lobby21.jpg" "/content"
!mv "/content/test/library/library_bookshelves_large.jpg" "/content"
!mv "/content/test/trainstation/gare_58_16_flickr.jpg" "/content"
!mv "/content/test/mall/West_End_Mall_3_DSC01801_m.jpg" "/content"

In [12]:
batch_size = 256


# Add our data-augmentation parameters to ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255.,
                                  #  rotation_range = 40,
                                  #  width_shift_range = 0.2,
                                  #  height_shift_range = 0.2,
                                  #  shear_range = 0.2,
                                  #  zoom_range = 0.2,
                                   horizontal_flip = True,
                                   )

test_datagen = ImageDataGenerator(rescale = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir, batch_size = batch_size, class_mode = 'categorical', target_size = (224, 224),)

validation_generator = test_datagen.flow_from_directory( validation_dir, batch_size = batch_size, class_mode = 'categorical', target_size = (224, 224),)


Found 12467 images belonging to 67 classes.
Found 3147 images belonging to 67 classes.


In [13]:
base_model = efn.EfficientNetB0(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')


for layer in base_model.layers:
    layer.trainable = False

16804768/16804768 [==============================] - 0s 0us/step


In [14]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dense(512, activation="relu")(x) # This is new edit of sequential
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(67, activation="softmax")(x)
model_final = tf.keras.models.Model(base_model.input, predictions)


In [15]:
model_final.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [16]:
eff_history = model_final.fit(train_generator, validation_data = validation_generator, steps_per_epoch = train_generator.samples//batch_size, epochs = 10)


Epoch 1/10
48/48 [==============================] - 171s 3s/step - loss: 3.2536 - accuracy: 0.3200 - val_loss: 1.7734 - val_accuracy: 0.5583
Epoch 2/10
48/48 [==============================] - 148s 3s/step - loss: 1.7561 - accuracy: 0.5515 - val_loss: 1.3616 - val_accuracy: 0.6244
Epoch 3/10
48/48 [==============================] - 148s 3s/step - loss: 1.2830 - accuracy: 0.6522 - val_loss: 1.2626 - val_accuracy: 0.6575
Epoch 4/10
48/48 [==============================] - 148s 3s/step - loss: 0.9855 - accuracy: 0.7263 - val_loss: 1.2230 - val_accuracy: 0.6644
Epoch 5/10
48/48 [==============================] - 157s 3s/step - loss: 0.7818 - accuracy: 0.7791 - val_loss: 1.2278 - val_accuracy: 0.6768
Epoch 6/10
48/48 [==============================] - 148s 3s/step - loss: 0.5936 - accuracy: 0.8282 - val_loss: 1.2586 - val_accuracy: 0.6918
Epoch 7/10
48/48 [==============================] - 158s 3s/step - loss: 0.4901 - accuracy: 0.8581 - val_loss: 1.2750 - val_accuracy: 0.6864
Epoch 8/10
48

In [ ]:
model_final.save('/content/drive/MyDrive/69prcnt_accuracy_efficientnet')


# Model

In [ ]:
https://drive.google.com/drive/folders/1FMdyCA4O4VA0NI7JzGEHztKm3cCGloUn?usp=sharing

In [25]:
!rm -r /content/drive/MyDrive/69prcnt_accuracy_efficientnet

# Prediction

In [18]:
# !mv "/content/test/classroom/Imagen_008.jpg" "/content"
# !mv "/content/test/gym/gimnasio_74_01_flickr.jpg" "/content"
# !mv "/content/test/lobby/lobby21.jpg" "/content"
# !mv "/content/test/library/library_bookshelves_large.jpg" "/content"
# !mv "/content/test/trainstation/gare_58_16_flickr.jpg" "/content"
# !mv "/content/test/mall/West_End_Mall_3_DSC01801_m.jpg" "/content"


In [19]:
# from tensorflow.keras.applications.efficientnet import EfficientNetB0

# # Load the EfficientNetB0 model with pre-trained weights
# model = EfficientNetB0(weights='imagenet')

In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


image_path = "/content/West_End_Mall_3_DSC01801_m.jpg"

img = load_img(image_path, target_size = (224, 224),)

img_array = img_to_array(img)
# Add batch dimension
img_array = np.expand_dims(img_array, axis=0)


datagen = ImageDataGenerator(rescale=1./255)
img_generator = datagen.flow(img_array, batch_size=20, )






# # Load and preprocess the image
# image = tf.keras.preprocessing.image.load_img('/content/Images/office/office20.jpg', target_size=(224, 224))
# image_array = tf.keras.preprocessing.image.img_to_array(image)
# preprocessed_image = ImageDataGenerator(rescale = 1.0/255.)


# # Reshape the preprocessed image if needed (e.g., for single prediction)
# input_image = tf.expand_dims(image_array, axis=0)


# Perform the prediction
predictions = model_final.predict(img_generator)



predicted_class = np.argmax(predictions, axis=1)
print(predicted_class)



# Map class index to class label using a dictionary
predicted_label = images_folder[predicted_class[0]]
print(predicted_label)

1/1 [==============================] - 2s 2s/step
[44]
dentaloffice
